In [1]:
from dotenv import find_dotenv
from dotenv import load_dotenv
print(find_dotenv())
succ = load_dotenv(find_dotenv())
if not succ:
    raise ValueError('load_dotenv FALSE')

/home/poyuan/workspace/rag30/.env


In [2]:
from llama_index.llms.openai import OpenAI

In [3]:
# set context for llm provider
llm = OpenAI(model="gpt-5-mini", temperature=0.0)

In [4]:
# read document
from llama_index.core import SimpleDirectoryReader
source_dir = '../data/'
reader = SimpleDirectoryReader(source_dir, required_exts=['.txt'])
documents = reader.load_data()

In [5]:
doc = documents[0]
doc.text[:100]

'各位同學大家好 我們來上課吧\n剛才只是用Google的VO3\n這個可以生成影片的人工智慧\n隨便生了一些影片\n作為開場,我們現在來正式開始這門課\n我們先把投影片開起來\n歡迎來到這門超級有趣的課程\n很高興'

In [6]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
# instantiate a DatasetGenerator
dataset_generator = RagDatasetGenerator.from_documents(
    documents,
    llm=llm,
    num_questions_per_chunk=1,  # set the number of questions per nodes
    show_progress=True,
)

/home/poyuan/miniconda3/envs/rag30/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parsing nodes: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.30it/s]


In [7]:
len(dataset_generator.nodes)

9

In [9]:
for idx, node in enumerate(dataset_generator.nodes):
    print(node.text[:10])
    print(f"idx: {idx}, beg: {node.start_char_idx}, end: {node.end_char_idx}")

各位同學大家好 我們
idx: 0, beg: 0, end: 720
第一種
如果你未來想
idx: 1, beg: 721, end: 1527
to this co
idx: 2, beg: 1436, end: 2255
code放在這頁投影
idx: 3, beg: 2256, end: 3003
Cool上留言區的問
idx: 4, beg: 2869, end: 3277
Cool寄給所有有修
idx: 5, beg: 3231, end: 4015
Cool那個平台上面
idx: 6, beg: 3882, end: 4612
業設計成這樣呢
為什
idx: 7, beg: 4482, end: 5182
預防針
幫助你在未來
idx: 8, beg: 5038, end: 5264


In [11]:
doc.text[721:731]

'第一種\n如果你未來想'

In [12]:
questions = dataset_generator.generate_questions_from_nodes()

100%|█████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:32<00:00,  3.62s/it]


In [14]:
df = questions.to_pandas()
df

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,請回答下列問題（每題請用完整句說明）：,[各位同學大家好 我們來上課吧\n剛才只是用Google的VO3\n這個可以生成影片的人工智...,,None,ai (gpt-5-mini)
1,根據本文內容完成下列題目：,[第一種\n如果你未來想要更深入人工智慧領域\n這門可絕對是很好的入門\n當然要完全掌握AI...,,None,ai (gpt-5-mini)
2,請完成下列題目（請以條列或短文回答）：,[to this course\n這個也是VO3自己生成的\n他產生的影片是帶有聲音的\n那...,,None,ai (gpt-5-mini)
3,題目：請根據課程說明回答下列問題（每題請簡潔說明）：,[code放在這頁投影片上面\n那我們上課的方式呢\n主要是以直播進行\n那我會在那個台灣大...,,None,ai (gpt-5-mini)
4,請根據上文回答下列四個問題並簡要說明理由或步驟：,[Cool上留言區的問題\n那旁聽生\n如果你想要加入NTU Cool的話\n請直接寄信給助...,,None,ai (gpt-5-mini)
5,請回答下列題目（分三部分）：,[Cool寄給所有有修課的同學\n但是假設你發現直播斷線\n非常重要的一點就是\n請勿驚慌 ...,,None,ai (gpt-5-mini)
6,閱讀上文後，請回答下列三部分的題目：,[Cool那個平台上面\n回答一些選擇題\n就算是完成作業的繳交\n那今天就會公告作業一\n...,,None,ai (gpt-5-mini)
7,請根據上文內容回答下列問題（單題多項，請依序作答）：,[業設計成這樣呢\n為什麼作業要跑三個小時呢\n我只允許這個作業三分鐘\n就應該要跑完\n我...,,None,ai (gpt-5-mini)
8,根據本文內容，請回答下列各項：,[預防針\n幫助你在未來面對更大的挑戰\n幫助你在未來面對挑戰的時候\n做好心理準備\n另外...,,None,ai (gpt-5-mini)


In [15]:
import os
DEST_DIR= 'questions'
os.makedirs(DEST_DIR, exist_ok=True)
df.to_json(
    os.path.join(DEST_DIR, "question_v1.json"),
    orient="records",
    force_ascii=False,
    indent=2
)

# v2

In [16]:
text_question_template = """以下是情境資訊：
---------------------
{context_str}
---------------------
請僅根據上述情境資訊（而非先驗知識），
依照下方查詢生成問題。
{query_str}
"""

question_gen_query = """你是一位老師／教授。你的任務是為即將到來的測驗／考試，根據所提供的內容資訊，設計 1 題考題。  
考題應該涵蓋文件中的多樣化主題，並且僅限於給定的情境資訊。"""



In [17]:
from llama_index.core.prompts.base import BasePromptTemplate, PromptTemplate

In [18]:
dataset_generator_v2 = RagDatasetGenerator.from_documents(
    documents,
    llm=llm,
    num_questions_per_chunk=1,  # set the number of questions per nodes
    text_question_template = PromptTemplate(text_question_template),
    question_gen_query = question_gen_query,
    show_progress=True,
)

Parsing nodes: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 112.82it/s]


In [19]:
questions_v2 = dataset_generator_v2.generate_questions_from_nodes()

  0%|                                                                                             | 0/9 [00:00<?, ?it/s]2025-09-19 22:16:20,491 - INFO - Retrying request to /chat/completions in 0.494151 seconds
2025-09-19 22:16:30,945 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
100%|█████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:40<00:00,  4.47s/it]


In [20]:
df = questions_v2.to_pandas()
df

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,請依據課堂導言（上述內容）作答。此題共三小題，請用完整句子說明你的答案。,[各位同學大家好 我們來上課吧\n剛才只是用Google的VO3\n這個可以生成影片的人工智...,,None,ai (gpt-5-mini)
1,考題（1題，含3部分，共20分）：,[第一種\n如果你未來想要更深入人工智慧領域\n這門可絕對是很好的入門\n當然要完全掌握AI...,,None,ai (gpt-5-mini)
2,出題（共 1 題，總分 15 分）：,[to this course\n這個也是VO3自己生成的\n他產生的影片是帶有聲音的\n那...,,None,ai (gpt-5-mini)
3,根據課堂說明，請回答下列各題（每小題詳述理由與根據課堂內容）：,[code放在這頁投影片上面\n那我們上課的方式呢\n主要是以直播進行\n那我會在那個台灣大...,,None,ai (gpt-5-mini)
4,請閱讀下列情境，並回答題中各小題（考試採簡答）：,[Cool上留言區的問題\n那旁聽生\n如果你想要加入NTU Cool的話\n請直接寄信給助...,,None,ai (gpt-5-mini)
5,請回答下列一題（總分 30 分）。請僅根據提供的情境資訊作答。,[Cool寄給所有有修課的同學\n但是假設你發現直播斷線\n非常重要的一點就是\n請勿驚慌 ...,,None,ai (gpt-5-mini)
6,請閱讀下列情境摘要（僅以考題所給資訊為依據），然後回答問題（共 25 分）：,[Cool那個平台上面\n回答一些選擇題\n就算是完成作業的繳交\n那今天就會公告作業一\n...,,None,ai (gpt-5-mini)
7,考題（共一題，分為四部分，請依題目中所給情境資訊作答）：,[業設計成這樣呢\n為什麼作業要跑三個小時呢\n我只允許這個作業三分鐘\n就應該要跑完\n我...,,None,ai (gpt-5-mini)
8,出題（1題，共100分）：,[預防針\n幫助你在未來面對更大的挑戰\n幫助你在未來面對挑戰的時候\n做好心理準備\n另外...,,None,ai (gpt-5-mini)


In [21]:
df.to_json(
    os.path.join(DEST_DIR, "question_v2.json"),
    orient="records",
    force_ascii=False,
    indent=2
)

# v3

In [24]:
num_questions_per_chunk = 1
text_question_template = """以下是上課內容片段：
---------------------
{context_str}
---------------------
{query_str}
"""

question_gen_query = f"""你是一位老師／教授。你的任務是為根據所提供的上課內容片段，設計 {num_questions_per_chunk} 題問題來問學生。  
問題應該涵蓋文件中的多樣化主題，並且需要在上課內容片段中找的到答案。"""

question_gen_query

'你是一位老師／教授。你的任務是為根據所提供的上課內容片段，設計 1 題問題來問學生。  \n問題應該涵蓋文件中的多樣化主題，並且需要在上課內容片段中找的到答案。'

In [25]:
dataset_generator_v3 = RagDatasetGenerator.from_documents(
    documents,
    llm=llm,
    num_questions_per_chunk=num_questions_per_chunk,  # set the number of questions per nodes
    text_question_template = PromptTemplate(text_question_template),
    question_gen_query = question_gen_query,
    show_progress=True,
)

Parsing nodes: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 107.77it/s]


In [26]:
questions_v3 = dataset_generator_v3.generate_questions_from_nodes()

  0%|                                                                                             | 0/9 [00:00<?, ?it/s]2025-09-19 22:25:07,564 - INFO - Retrying request to /chat/completions in 0.390820 seconds
2025-09-19 22:25:16,084 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
 11%|█████████▍                                                                           | 1/9 [00:08<01:08,  8.53s/it]2025-09-19 22:25:16,099 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-19 22:25:17,025 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
100%|█████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:25<00:00,  2.84s/it]


In [27]:
df = questions_v3.to_pandas()
df

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,請回答下列三小題（答案均可從課堂片段中找到）：,[各位同學大家好 我們來上課吧\n剛才只是用Google的VO3\n這個可以生成影片的人工智...,,None,ai (gpt-5-mini)
1,請根據上課內容片段回答以下問題（可各列要點）：,[第一種\n如果你未來想要更深入人工智慧領域\n這門可絕對是很好的入門\n當然要完全掌握AI...,,None,ai (gpt-5-mini)
2,請回答下列問題（可分段作答）：,[to this course\n這個也是VO3自己生成的\n他產生的影片是帶有聲音的\n那...,,None,ai (gpt-5-mini)
3,請根據課堂說明回答下列各題（在課堂內容片段中可找到答案）：,[code放在這頁投影片上面\n那我們上課的方式呢\n主要是以直播進行\n那我會在那個台灣大...,,None,ai (gpt-5-mini)
4,請回答下列綜合題（可分項作答）：,[Cool上留言區的問題\n那旁聽生\n如果你想要加入NTU Cool的話\n請直接寄信給助...,,None,ai (gpt-5-mini)
5,請回答下列問題（所有答案都可在上課內容片段中找到）：,[Cool寄給所有有修課的同學\n但是假設你發現直播斷線\n非常重要的一點就是\n請勿驚慌 ...,,None,ai (gpt-5-mini)
6,請讀取課堂片段後回答下列問題（答案均可在片段中找到）：,[Cool那個平台上面\n回答一些選擇題\n就算是完成作業的繳交\n那今天就會公告作業一\n...,,None,ai (gpt-5-mini)
7,請根據上課內容片段回答下列問題：,[業設計成這樣呢\n為什麼作業要跑三個小時呢\n我只允許這個作業三分鐘\n就應該要跑完\n我...,,None,ai (gpt-5-mini)
8,請回答下列問題（可分點作答）：,[預防針\n幫助你在未來面對更大的挑戰\n幫助你在未來面對挑戰的時候\n做好心理準備\n另外...,,None,ai (gpt-5-mini)


In [28]:
df.to_json(
    os.path.join(DEST_DIR, "question_v3.json"),
    orient="records",
    force_ascii=False,
    indent=2
)

# v4

In [29]:
num_questions_per_chunk = 1
text_question_template = """以下是上課內容片段：
---------------------
{context_str}
---------------------
{query_str}
"""

question_gen_query = f"""你的任務是為根據所提供的上課內容片段，設計 {num_questions_per_chunk} 題問題來問學生。  
問題應該涵蓋文件中的多樣化主題，並且需要在上課內容片段中找的到答案。
範例輸出：
什麼是深度學習的醍醐味？
"""



In [31]:
dataset_generator_v4 = RagDatasetGenerator.from_documents(
    documents,
    llm=llm,
    num_questions_per_chunk = num_questions_per_chunk,  # set the number of questions per nodes
    text_question_template = PromptTemplate(text_question_template),
    question_gen_query = question_gen_query,
    show_progress=True,
)

Parsing nodes: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 115.41it/s]


In [32]:
questions_v4 = dataset_generator_v4.generate_questions_from_nodes()

  0%|                                                                                             | 0/9 [00:00<?, ?it/s]2025-09-19 22:26:39,966 - INFO - Retrying request to /chat/completions in 0.392901 seconds
2025-09-19 22:26:47,428 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
 78%|██████████████████████████████████████████████████████████████████                   | 7/9 [00:20<00:05,  2.53s/it]2025-09-19 22:27:03,392 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-19 22:27:03,394 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
100%|█████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:23<00:00,  2.60s/it]


In [33]:
df = questions_v4.to_pandas()
df

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,這門課的主要目標是什麼？老師用什麼比喻來說明為何要了解AI背後的原理，並簡述該比喻的重點？,[各位同學大家好 我們來上課吧\n剛才只是用Google的VO3\n這個可以生成影片的人工智...,,None,ai (gpt-5-mini)
1,請列出上課片段中提到的兩個AI工具（VO3 與 Eleven Labs），分別說明它們在示範...,[第一種\n如果你未來想要更深入人工智慧領域\n這門可絕對是很好的入門\n當然要完全掌握AI...,,None,ai (gpt-5-mini)
2,這門課共有幾講？請說明前四講、第五到第八講與第九、第十講各自的重點主題；並說明老師示範影片中...,[to this course\n這個也是VO3自己生成的\n他產生的影片是帶有聲音的\n那...,,None,ai (gpt-5-mini)
3,請說明本課程每週的上課時間與流程（包含授課與助教講解作業的大致時長）、課程錄影何時會上線，以...,[code放在這頁投影片上面\n那我們上課的方式呢\n主要是以直播進行\n那我會在那個台灣大...,,None,ai (gpt-5-mini)
4,如果你想要旁聽這門課，要如何加入 NTU Cool？在 NTU Cool 的哪裡可以留下上課...,[Cool上留言區的問題\n那旁聽生\n如果你想要加入NTU Cool的話\n請直接寄信給助...,,None,ai (gpt-5-mini)
5,請說明本課程作業的時程與繳交方式：每份作業通常給多少時間？前兩個作業的截止日為何？最後一個作...,[Cool寄給所有有修課的同學\n但是假設你發現直播斷線\n非常重要的一點就是\n請勿驚慌 ...,,None,ai (gpt-5-mini)
6,除了在 NTU Cool 繳交作業外，還有哪個繳交系統？它的用途是什麼？另外，課程把作業難度...,[Cool那個平台上面\n回答一些選擇題\n就算是完成作業的繳交\n那今天就會公告作業一\n...,,None,ai (gpt-5-mini)
7,為什麼講者會在課程中保留需要訓練三到四小時的作業？請說明至少兩個理由，並指出課程目前是否包含...,[業設計成這樣呢\n為什麼作業要跑三個小時呢\n我只允許這個作業三分鐘\n就應該要跑完\n我...,,None,ai (gpt-5-mini)
8,如果你對生成式AI一無所知，老師建議先看哪一門課程？看完之後想進一步了解這些AI是如何被訓練...,[預防針\n幫助你在未來面對更大的挑戰\n幫助你在未來面對挑戰的時候\n做好心理準備\n另外...,,None,ai (gpt-5-mini)


In [34]:
df.to_json(
    os.path.join(DEST_DIR, "question_v4.json"),
    orient="records",
    force_ascii=False,
    indent=2
)

In [35]:
questions.save_json?

Signature: questions.save_json(path: str) -> None
Docstring: Save json.
File:      ~/miniconda3/envs/rag30/lib/python3.12/site-packages/llama_index/core/llama_dataset/base.py
Type:      method

# v5

In [41]:
num_questions_per_chunk = 1

text_question_template = PromptTemplate(
    f"""你是一位專業的課程助教，你的任務是根據上課內容的片段，設計 {num_questions_per_chunk} 個問題來問學生。
問題風格以單一且直接的問句為主，並且需要在上課內容中找得到答案。
若是要求出複數題，則問題應該盡可能涵蓋不同的面向。

範例輸入:
以下是上課內容片段：
---------------------
"業設計成這樣呢\n為什麼作業要跑三個小時呢\n我只允許這個作業三分鐘\n就應該要>跑完\n我們有當然可以把這些特別花時間\n需要特別花時間訓練的作業拿掉\n但是我還是選>擇在這門課裡面\n保留那一些\n需要一定訓練時間的作業\n因為焦躁的等待人工智慧訓練的結果\n迷茫的調參數\n不知道會不會成功\n這個就是人工智慧的醍醐味\n所以大家需要學習\n在迷茫中前進\n這個就是模型的訓練\n我們特別把這一部分保留在課程裡面\n讓你體驗說\n模型訓練不出來的焦躁\n到底是什麼樣的感覺\n而且我必須要強調啊\n什麼三四個小時的等待訓練時間\n真的不算什麼\n我們過去有很多作業訓練時間都是\n至少三天起跳\n你要至少訓練三天\n你才能夠拿到成績\n那我知道說很多同學在做作業的時候\n往往你都在實現\n最後一天才開始做作業\n但那一種啊\n需要訓練三天以上的模型\n你只在前一天才開始做作業\n你是絕不可能完成的\n這個時候我告訴你\n你唯一可以做的事情\n就是放棄這樣子\n還好我們這堂課裡面呢\n現在是沒有需要訓練一天以上的作業啦\n我們把那種\n特別需要花時間訓練的作業\n還是拿掉了\n只保留了需要訓練三四個小時的作業\n但我只想要強調說\n三四個小時的訓練時間\n真的不算什麼\n如果你要真正用大量的資料\n大規模的訓練模型\n訓練個數週\n其實都是常見的事情\n而這一些\n需要一點訓練時間的作業\n它的定位就像是預防針\n幫助你在未來面對更大的挑戰\n幫助你在未來面對挑戰的時候\n做好心理準備\n另外呢\n鼓勵大家如果有空的話\n可以先看一些線上的錄影進行預習\n那假設你對於生成式AI一無所知的話\n那你可以先看生成式AI導論2024的課程\n那如果2024的課程看完\n你想要進一步了解\n這些AI是怎麼被訓練出來的"
---------------------
範例輸出:
什麼是深度學習的醍醐味？

輸入:
以下是上課內容片段：
---------------------
{{context_str}}
---------------------
"""
)

question_gen_query = '輸出:\n'

In [42]:
dataset_generator_v5 = RagDatasetGenerator.from_documents(
    documents,
    llm=llm,
    num_questions_per_chunk = num_questions_per_chunk,  # set the number of questions per nodes
    text_question_template = text_question_template,
    question_gen_query = question_gen_query,
    show_progress=True,
)

Parsing nodes: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 109.81it/s]


In [43]:
questions_v5 = dataset_generator_v5.generate_questions_from_nodes()
df = questions_v5.to_pandas()
df

  0%|                                                                                             | 0/9 [00:00<?, ?it/s]2025-09-19 23:04:32,700 - INFO - Retrying request to /chat/completions in 0.473507 seconds
2025-09-19 23:04:37,650 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
100%|█████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:18<00:00,  2.08s/it]


,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,剛才老師用哪一個可以生成影片的人工智慧做開場？,[各位同學大家好 我們來上課吧\n剛才只是用Google的VO3\n這個可以生成影片的人工智...,,None,ai (gpt-5-mini)
1,VO3可以接受哪些輸入來生成影片？,[第一種\n如果你未來想要更深入人工智慧領域\n這門可絕對是很好的入門\n當然要完全掌握AI...,,None,ai (gpt-5-mini)
2,講者用哪個軟體合成他的講課聲音？,[to this course\n這個也是VO3自己生成的\n他產生的影片是帶有聲音的\n那...,,None,ai (gpt-5-mini)
3,這門課每週的上課時間何時開始？,[code放在這頁投影片上面\n那我們上課的方式呢\n主要是以直播進行\n那我會在那個台灣大...,,None,ai (gpt-5-mini)
4,如果你想加入 NTU Cool 當旁聽生，應該怎麼做？,[Cool上留言區的問題\n那旁聽生\n如果你想要加入NTU Cool的話\n請直接寄信給助...,,None,ai (gpt-5-mini)
5,這門課總共有幾講？,[Cool寄給所有有修課的同學\n但是假設你發現直播斷線\n非常重要的一點就是\n請勿驚慌 ...,,None,ai (gpt-5-mini)
6,JudgeBoi 是什麼？,[Cool那個平台上面\n回答一些選擇題\n就算是完成作業的繳交\n那今天就會公告作業一\n...,,None,ai (gpt-5-mini)
7,講者為什麼在課程中保留那些需要一定訓練時間的作業？,[業設計成這樣呢\n為什麼作業要跑三個小時呢\n我只允許這個作業三分鐘\n就應該要跑完\n我...,,None,ai (gpt-5-mini)
8,如果你對生成式AI一無所知，講者建議先看哪一門課程？,[預防針\n幫助你在未來面對更大的挑戰\n幫助你在未來面對挑戰的時候\n做好心理準備\n另外...,,None,ai (gpt-5-mini)


In [44]:
df.to_json(
    os.path.join(DEST_DIR, "question_v5.json"),
    orient="records",
    force_ascii=False,
    indent=2
)